In [1]:
import os
import sys
import findspark
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901 pyspark-shell'
# Set Spark Home manually
os.environ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec"
os.environ["PYSPARK_PYTHON"] = "python3"




In [2]:
# Ensure Python can find Spark
sys.path.append("/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/python/lib")

findspark.init()
print("findspark initialized successfully!")

findspark initialized successfully!


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session started successfully!")
spark


In [3]:

from pyspark.sql import SparkSession
from py4j.java_gateway import java_import


spark = SparkSession.builder \
    .appName("IcebergTableSetup") \
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.my_catalog.type", "hadoop") \
    .config("spark.sql.catalog.my_catalog.warehouse", os.environ.get("s3_location")) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", os.environ.get("AWS_ACCESS_KEY_ID")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ.get("AWS_SECRET_ACCESS_KEY")) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .config("spark.sql.adaptive.skewJoin.enabled", "true") \
                .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
                    .getOrCreate()

df = spark.read.csv("s3a://nonprofit-explorer/raw_data/eo1.csv", header=True)
df.show(5)

25/03/08 20:00:36 WARN Utils: Your hostname, Sarthaks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.20.20 instead (on interface en0)
25/03/08 20:00:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/sarthak/.ivy2/cache
The jars for the packages stored in: /Users/sarthak/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b932eb86-b653-410e-835c-ba8c222ba844;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 96ms :: artifacts dl 3ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-b932eb86-b653-410e-835c-ba8c222ba844
	confs: [default]
	

+---------+--------------------+---------------+--------+-------------+-----+----------+-----+----------+-----------+--------------+------+-------------+----------+---------+------------+------+----------+--------+---------+-------------+----------------+-------+---------+----------+-----------+-------+---------+
|      EIN|                NAME|            ICO|  STREET|         CITY|STATE|       ZIP|GROUP|SUBSECTION|AFFILIATION|CLASSIFICATION|RULING|DEDUCTIBILITY|FOUNDATION| ACTIVITY|ORGANIZATION|STATUS|TAX_PERIOD|ASSET_CD|INCOME_CD|FILING_REQ_CD|PF_FILING_REQ_CD|ACCT_PD|ASSET_AMT|INCOME_AMT|REVENUE_AMT|NTEE_CD|SORT_NAME|
+---------+--------------------+---------------+--------+-------------+-----+----------+-----+----------+-----------+--------------+------+-------------+----------+---------+------------+------+----------+--------+---------+-------------+----------------+-------+---------+----------+-----------+-------+---------+
|010215913|BUCK MEMORIAL LIB...|    % TREASURER| MAIN S

In [5]:
df

DataFrame[EIN: string, NAME: string, ICO: string, STREET: string, CITY: string, STATE: string, ZIP: string, GROUP: string, SUBSECTION: string, AFFILIATION: string, CLASSIFICATION: string, RULING: string, DEDUCTIBILITY: string, FOUNDATION: string, ACTIVITY: string, ORGANIZATION: string, STATUS: string, TAX_PERIOD: string, ASSET_CD: string, INCOME_CD: string, FILING_REQ_CD: string, PF_FILING_REQ_CD: string, ACCT_PD: string, ASSET_AMT: string, INCOME_AMT: string, REVENUE_AMT: string, NTEE_CD: string, SORT_NAME: string]

S3A Implementation: org.apache.hadoop.fs.s3a.S3AFileSystem


In [7]:
s3_path = "s3a://nonprofit-explorer/raw_data/"

# Load CSVs from S3
df = spark.read.option("header", True).csv(s3_path)

# Show data
df.show()


+---------+--------------------+-------------------+--------------------+--------------+-----+----------+-----+----------+-----------+--------------+------+-------------+----------+---------+------------+------+----------+--------+---------+-------------+----------------+-------+---------+----------+-----------+-------+--------------------+
|      EIN|                NAME|                ICO|              STREET|          CITY|STATE|       ZIP|GROUP|SUBSECTION|AFFILIATION|CLASSIFICATION|RULING|DEDUCTIBILITY|FOUNDATION| ACTIVITY|ORGANIZATION|STATUS|TAX_PERIOD|ASSET_CD|INCOME_CD|FILING_REQ_CD|PF_FILING_REQ_CD|ACCT_PD|ASSET_AMT|INCOME_AMT|REVENUE_AMT|NTEE_CD|           SORT_NAME|
+---------+--------------------+-------------------+--------------------+--------------+-----+----------+-----+----------+-----------+--------------+------+-------------+----------+---------+------------+------+----------+--------+---------+-------------+----------------+-------+---------+----------+-----------+-

In [ ]:
import pyspark
print(pyspark.__version__)
